In [1]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
import re
from urllib import urlopen
url = "https://www.tripadvisor.com"
html = urlopen(url)
bsObj = BeautifulSoup(html.read(),"lxml")

In [2]:
html = urlopen(url+"/Hotels-g30196-Austin_Texas-Hotels.html")
bsObj = BeautifulSoup(html.read(),"lxml")
num = 0
urlList = []
while num<=6:
    divList = bsObj.findAll("div",{"class":'prw_rup prw_meta_short_cell_listing'})
    for i in divList:
        urlList.append(i.find("a")['href'])
    num = num+1
    html = urlopen(url+'/Hotels-g30196-oa'+str(num*30)+'-Austin_Texas-Hotels.html')
    bsObj = BeautifulSoup(html.read(),"lxml")
urlSet = set(urlList)

In [3]:
keys = ['Excellent','Very good','Average','Poor','Terrible','Families','Couples','Solo','Business',
        'Friends','Mar-May','Jun-Aug','Sep-Nov','Dec-Feb','English']

In [4]:
dictionary = {}
for i in keys:
    dictionary[i] = []
dictionary['hotels'] =[]
dictionary['reveiwCount'] =[]

for u in urlSet:
    html = urlopen(url+u)
    bsObj = BeautifulSoup(html.read(),"lxml")
    try:
        if eval(bsObj.find("a",{'property':"reviewCount"})['content'].encode('utf-8'))>=50:
            dictionary['hotels'].append(u.split("Reviews-")[1].split("Austin_Texas")[0])
            temp_dict={}
            dictionary['reveiwCount'].append(bsObj.find("a",{'property':"reviewCount"})['content'].encode('utf-8'))
            for i in bsObj.findAll("span",{'class':"toggle"}):
                temp  = i.findNext().findNext().get_text().encode('utf-8').strip().replace("\n\n\n\n","(").strip(")").split("(")
                if temp[0].strip() in keys:
                    temp_dict[temp[0].strip()] = temp[1]
            for i in keys:
                try:
                    dictionary[i].append(temp_dict[i])
                except:
                    dictionary[i].append(0)
    except:
        print u

In [17]:
import pandas as pd

df = pd.DataFrame.from_dict(dictionary)
##df = df.set_index('hotels')
#df = df.reindex(columns=keys.append('reviewCount'))
#df = df.reset_index()
df['hotels'] = df['hotels'].str.strip("-")
df.hotels.nunique()

87

In [6]:
df.to_pickle('taNumeric.pkl')

In [28]:
df.to_csv('tripAd.csv',encoding='utf-8')
df.head()

,hotels,Average,Business,Couples,Dec-Feb,English,Excellent,Families,Friends,Jun-Aug,Mar-May,Poor,Sep-Nov,Solo,Terrible,Very good,reveiwCount
0,La_Quinta_Inn_Suites_Austin_Mopac_North,52,128,48,75,342,134,114,10,98,86,34,83,25,22,100,345
1,Sleep_Inn_Suites,12,29,43,31,168,105,55,14,53,42,1,42,9,2,48,174
2,La_Quinta_Inn_Austin_North,56,132,57,73,351,150,91,33,109,87,27,82,20,21,97,352
3,Hampton_Inn_Suites_Austin_at_The_University_Ca...,47,345,239,292,"1,165",888,369,86,332,280,10,261,62,4,216,1181
4,Hotel_Saint_Cecilia,6,14,66,33,124,87,16,12,40,32,3,19,11,1,27,125
